# 加标签 Tagging

## 文本分类到标签 Classify Text into Labels

标记意味着使用类别（如以下）来标注文档：

 - 情感
 - 语言
 - 风格（正式、非正式等）
 - 涉及的主题
 - 政治倾向

## 概述
标记包含几个组成部分：

 - 功能：与提取类似，标记使用函数来指定模型应如何标记文档
 - 架构：定义我们希望如何标记文档
## 快速入门
让我们来看一个非常直接的例子，展示如何在LangChain中使用OpenAI工具调用来进行标记。  
我们将使用OpenAI模型支持的with_structured_output方法：



In [1]:
import os
from dotenv import find_dotenv,load_dotenv

_ = load_dotenv(find_dotenv())

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel,Field
from langchain_openai import ChatOpenAI

tagging_prompt = ChatPromptTemplate.from_template("""
请提供一段具体的文本内容以便我从中抽取信息。
仅提取“Classification”函数中提到的属性。

段落:
{input}
""")

class Classification(BaseModel):
    # sentiment 情绪
    sentiment:str = Field(description="文本的情感")
    aggressiveness:int = Field(description="文本的攻击性程度在1到10的范围内是如何的")
    language:str = Field(description="文本使用的什么语言")

llm = ChatOpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4",
    api_key=os.environ["ZHIPUAI_API_KEY"],
    model="glm-4",
).with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

In [11]:
# 我很高兴认识你！我想我们会成为非常好的朋友
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain.invoke({"input": inp})

Classification(sentiment='积极', aggressiveness=1, language='西班牙语')

如果我们想要JSON输出，我们可以直接调用.dict（）

In [12]:
# 我对你很生气！我会给你应得的！
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = tagging_chain.invoke({"input": inp})
res.dict()

{'sentiment': 'negative', 'aggressiveness': 8, 'language': 'Spanish'}

正如我们在例子中看到的，它正确地解释了我们想要什么。

结果各不相同，因此我们可以获得不同语言的情感（“积极”、“enojado”等）。


## 更精细的控制
仔细的模式定义使我们能够更好地控制模型的输出。

具体来说，我们可以定义：

 - 每个属性的可能值
 - 描述以确保模型了解属性
 - 需要返回的属性
   
让我们重新声明我们的Pydantic模型，以使用枚举来控制前面提到的每个方面：

In [13]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "chinese"]
    )

In [14]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = ChatOpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4",
    api_key=os.environ["ZHIPUAI_API_KEY"],
    model="glm-4",
).with_structured_output(
    Classification
)

chain = tagging_prompt | llm

In [21]:
inp = "张三在这场比赛中真是yyds!"
chain.invoke({"input": inp})

Classification(sentiment='happy', aggressiveness=1, language='chinese')

In [20]:
inp = "咱们大概八字不合吧"
chain.invoke({"input": inp})

Classification(sentiment='sad', aggressiveness=2, language='chinese')

In [23]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.invoke({"input": inp})

Classification(sentiment='neutral', aggressiveness=1, language='english')